Importing relevant packages and modules

In [1]:
import sys
from pathlib import Path

# Plotting
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import gaussian_kde

'''
CUSTOM PACKAGES
'''
# avoid adding multiple relave paths to sys.path
sys.path.append("../src") if "../src" not in sys.path else None

from Experiment import Experiment
from stats import grid_score_masks, get_smooth_ratemaps, grid_orientation, grid_spacing, apply_to_selection, circular_kernel

TypeError: 'ABCMeta' object is not subscriptable

Load the Experiment data

In [ ]:
base_path = Path("/storA/GridCells/")
experiment = Experiment(name="3ME", base_path=base_path)
experiment.setup()

Create the masks based on specified selection scheme (grid score, topology, ...)

In [ ]:
gs_masks = grid_score_masks(experiment, percentile=0.4, mode="intersection")

## Plotting

Plotting style related settings. Loading notebook stylesheet and setting the colors of the cycler. Colors are either based on the officiel BioAI color scheme or on the diverging color scheme from [ColorBrewer2](https://colorbrewer2.org/#type=diverging&scheme=BrBG&n=3).

In [ ]:
plt.style.use(['seaborn-paper', './notebook.mplstyle'])

# BioAI color scheme
bioAI_dark_blue = '#194587'
bioAI_bright_blue = '#1ca6e6'
bioAI_background_grey = '#e8e8e8'
bioAI_colors = [bioAI_dark_blue, bioAI_bright_blue, bioAI_background_grey]

# ColorBrewer diverging
gold = '#d8b365'
off_white = '#f5f5f5'
tyrkis = '#5ab4ac'
cb_diverge = [gold, off_white, tyrkis]

color_scheme=bioAI_colors

Retrieve and plot the different statistics based on smoothed versions of the ratemaps corresponding to the loaded experiment


In [ ]:
smooth_ratemaps = get_smooth_ratemaps(experiment, sigma=1.0)

### Orientations

In [ ]:
orientations = apply_to_selection(grid_orientation, smooth_ratemaps, gs_masks)

In [ ]:
# von Mises KDE parameter
kappa = 50.0

thetas = np.linspace(-np.pi, +np.pi, num=1001)

fig, ax = plt.subplots()
patches = []
for env_i, orientations_env in enumerate(orientations):
    # Kernel density estimation with periodic gaussian kernel (von Mises kernel)
    # because orientation is 2-pi periodic
    von_mises_pdf = circular_kernel(orientations_env, kappa=kappa)
    ax.plot(
        thetas, 
        von_mises_pdf(thetas), 
        color=color_scheme[env_i], 
        linewidth=1.0,
        zorder=2
    ) # color
    ax.plot(
        thetas, 
        von_mises_pdf(thetas), 
        color='#000000', 
        linewidth=1.5,
        zorder=1
    )
    patches.append(
        mpatches.Patch(
            facecolor=color_scheme[env_i],
            label=r"$" + str(env_i) + "$",
            linewidth=0.5,
            edgecolor='#000000'
        )
    )
ymin, ymax = ax.get_ylim()
ax.set_ylim((ymin, ymax))
ax.vlines(
    np.array(
        [((60.0*i)*np.pi / 180.0) - np.pi for i in range(6)]
    ),
    ymin=ymin,
    ymax=ymax,
    color='grey',
    linestyles='dotted'
)
ax.set_ylim(bottom=0)
ax.set_ylabel(r"Density")
ax.set_xlabel(r"Orientation [rad]")
ax.legend(
    handles=patches,
    loc="upper right",
    title=r"Environment"
)
plt.show()

### Spacings

In [ ]:
spacings_list = apply_to_selection(grid_spacing, smooth_ratemaps, gs_masks)

In [ ]:
# kde parameter
bw_method = 0.5

fig, ax = plt.subplots()
patches=[]
for env_i, spacings_env in enumerate(spacings_list):
    # Kernel density estimation
    spacing_pdf = gaussian_kde(spacings_env, bw_method=bw_method)
    spacings = np.linspace(np.amin(spacings_env), np.amax(spacings_env), num=1001)
    ax.plot(
        spacings, 
        spacing_pdf(spacings),
        color=color_scheme[env_i],
        linewidth=1.0,
        zorder=2
    )
    ax.plot(
        spacings,
        spacing_pdf(spacings),
        color='#000000',
        linewidth=1.5,
        zorder=1
    )
    patches.append(
        mpatches.Patch(
            facecolor=color_scheme[env_i],
            label=r"$" + str(env_i) + "$",
            linewidth=0.5,
            edgecolor='#000000'
        )
    )
ax.set_ylim(bottom=0)
ax.set_ylabel(r"Density")
ax.set_xlabel(r"Spacing [px]")
ax.legend(
    handles=patches,
    loc="upper right",
    title=r"Environment"
)

# Phase Shifts

In [ ]:
phase_list = apply_to_selection(phase_shift, smooth_ratemaps, gs_masks)